In [6]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np

import warnings 
warnings.filterwarnings("ignore")

In [2]:
base_path = '/panfs/jay/groups/13/dizhu/xion2613/'

# Visitor census in one month -- visitor based

In [3]:
city_list = ['LA','Houston','Atlanta','Miami','Seattle','Denver','TCMA']
for city_name in city_list:
    visitor_cbg_four = pd.read_csv(base_path + 'Visitor_residents/social/' + city_name + '_social_four.csv')
    MSA_cbg = gpd.read_file('./MSA_cbg/' + city_name + '_cbg.geojson')
    MSA_cbg_re = MSA_cbg[['GEOID', 'GISJOIN']]
    MSA_cbg_re.loc[:, 'GEOID'] = MSA_cbg_re['GEOID'].astype(int)
    visitor_cbg_four = visitor_cbg_four.loc[:, ~visitor_cbg_four.columns.str.contains('^Unnamed')]
    visitor_cbg_four_re = visitor_cbg_four.rename({
                     "v_income": "Income_v", "v_wr": "Wp_v",
                     "v_pbb": "Pbb_v", "v_upr": "Upp_v"},  axis='columns')
    
    visitor_cbg_four_3 = MSA_cbg_re.merge(visitor_cbg_four_re, left_on='GEOID', right_on='CBG_target', how='left')
    visitor_cbg_four_out = visitor_cbg_four_3.drop(columns=['CBG_target','GEOID'])
    visitor_cbg_four_out = visitor_cbg_four_out.rename({"GISJOIN": "CBG_id_v"},  axis='columns')
    visitor_cbg_four_out.to_csv(base_path + 'Visitor_residents/Output/' + city_name + '_visitor_July2021.csv')
    print(len(visitor_cbg_four_out))

8706
4227
4058
3964
2765
2113
2641


# Visitor census in one month -- visit based

In [ ]:
city_list = ['LA','Houston','Atlanta','Miami','Seattle','Denver','TCMA']
for city_name in city_list:
    df_d = dd.read_csv(base_path + 'Visitor_residents/Visitor_census/' + city_name + '_visitor_census.csv')
    join_result_visitor_tab = df_d.compute()
    join_result_visitor_tab['CBG_home'] = join_result_visitor_tab['CBG_home'].astype(int)
    
    if city_name != 'TCMA':
        city_social_tab = pd.read_csv(base_path + 'Visitor_residents/social/' + city_name + '_social_tab_more.csv')
        city_social_tab['TL_GEO_ID'] = city_social_tab['TL_GEO_ID'].astype(int)
        join_result_visitor_tab = join_result_visitor_tab.merge(
                    city_social_tab[['TL_GEO_ID', 'ASP3E001','ASP3E022','ASP3E023','ASP3E024','ASP3E025',
                                  'ASSRE001', 'ASSRE005']], \
                    left_on='CBG_home', right_on='TL_GEO_ID', how='left')
        
        join_result_visitor_tab = join_result_visitor_tab.rename({"v_income": "Income_v", "v_pop": "pop_v", 
                 "v_white": "white_v", "v_black": "black_v", "v_asian": "asian_v",
                 "ASP3E001": "degree_v", "ASP3E022": "bac_v", "ASP3E023": "mas_v",
                 "ASP3E024": "pro_v", "ASP3E025": "doc_v",
                 "ASSRE001": "lab_v", "ASSRE005": "um_v"}, axis='columns')
    else:
        join_result_visitor_tab = join_result_visitor_tab.rename({"AQP6E001": "Income_v", "U7J001": "pop_v", 
                         "U7J002": "white_v", "U7J003": "black_v", "U7J005": "asian_v",
                         "ASP3E001": "degree_v", "ASP3E022": "bac_v", "ASP3E023": "mas_v",
                         "ASP3E024": "pro_v", "ASP3E025": "doc_v",
                         "ASSRE001": "lab_v", "ASSRE005": "um_v"}, axis='columns')
    
    visitor_cbg_income = join_result_visitor_tab.groupby('CBG_target')['Income_v'].mean().reset_index() # income

    join_result_visitor_tab['Wp_v'] = join_result_visitor_tab['white_v'] / (join_result_visitor_tab['pop_v']+0.00001)
    visitor_cbg_white = join_result_visitor_tab.groupby('CBG_target')['Wp_v'].mean().reset_index() # white ratio

    join_result_visitor_tab['Pbb_v'] = join_result_visitor_tab[['bac_v','mas_v','pro_v','doc_v']].sum(axis=1) / (join_result_visitor_tab['degree_v']+0.00001)
    visitor_cbg_pbb = join_result_visitor_tab.groupby('CBG_target')['Pbb_v'].mean().reset_index()  #ppb

    join_result_visitor_tab['Upp_v'] = join_result_visitor_tab['um_v'] / (join_result_visitor_tab['lab_v']+0.00001)
    visitor_cbg_upr = join_result_visitor_tab.groupby('CBG_target')['Upp_v'].mean().reset_index()  # upr

    visitor_cbg_four = visitor_cbg_income.merge(visitor_cbg_white, on='CBG_target', how='inner') \
                                  .merge(visitor_cbg_pbb, on='CBG_target', how='inner') \
                                  .merge(visitor_cbg_upr, on='CBG_target', how='inner')    
    
    MSA_cbg = gpd.read_file('./MSA_cbg/' + city_name + '_cbg.geojson')
    MSA_cbg_re = MSA_cbg[['GEOID', 'GISJOIN']]
    MSA_cbg_re.loc[:, 'GEOID'] = MSA_cbg_re['GEOID'].astype(int)
    visitor_cbg_four = visitor_cbg_four.loc[:, ~visitor_cbg_four.columns.str.contains('^Unnamed')]

    visitor_cbg_four_3 = MSA_cbg_re.merge(visitor_cbg_four, left_on='GEOID', right_on='CBG_target', how='left')
    visitor_cbg_four_out = visitor_cbg_four_3.drop(columns=['CBG_target','GEOID'])
    visitor_cbg_four_out = visitor_cbg_four_out.rename({"GISJOIN": "CBG_id_v"},  axis='columns')
    #visitor_cbg_four_out.to_csv(base_path + 'Visitor_residents/Output/visitation_based/' \
                                #+ city_name + '_visit_July2021.csv')
    print(len(visitor_cbg_four_out))

8706
4227
4058
